### Import the packages

In [1]:
import numpy as np
import pandas as pd
import glob
import os
import matplotlib.pyplot as plt
import string
import cv2
from mlxtend.plotting import plot_decision_regions
from mpl_toolkits.mplot3d import Axes3D

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.utils.multiclass import unique_labels
from sklearn import metrics
from sklearn.svm import SVC
import warnings
warnings.filterwarnings('ignore')

In [2]:
print(os.listdir("C://Users/Ishan.Borkar/Desktop/Tomato analysis/Kaggle/"))
dim = 1

['Testing', 'Training']


In [3]:
def getTomatoes(tomatoes, data_type, print_n=False, k_fold=False):
    images = []
    labels = []
    val = ['Training', 'Testing']
    if not k_fold:
        path = "C://Users/Ishan.Borkar/Desktop/Tomato analysis/Kaggle/" + data_type + "/"
        for i,f in enumerate(tomatoes):
            p = path + f
            j=0
            for image_path in glob.glob(os.path.join(p, "*.jpg")):
                image = cv2.imread(image_path, cv2.IMREAD_COLOR)
                image = cv2.resize(image, (dim, dim))
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
                images.append(image)
                labels.append(i)
                j+=1
            if(print_n):
                print("There are " , j , " " , data_type, "images of " , tomatoes[i])
        labels = np.array(labels)
        return images, labels
    else:
        for v in val:
            path = "C://Users/Ishan.Borkar/Desktop/Tomato analysis/Kaggle/" + v + "/"
            for i,f in enumerate(tomatoes):
                p = path + f
                j=0
                for image_path in glob.glob(os.path.join(p, "*.jpg")):
                    image = cv2.imread(image_path, cv2.IMREAD_COLOR)
                    image = cv2.resize(image, (dim, dim))
                    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
                    images.append(image)
                    labels.append(i)
                    j+=1
        images = np.array(images)
        labels = np.array(labels)
        return images, labels
    
def getAllTomatoes(tomatoes, data_type):
    tomatoes = []
    for tomato_path in glob.glob("C://Users/Ishan.Borkar/Desktop/Tomato analysis/Kaggle/Training/*"):
        tomato = tomato_path.split("/")[-1]
        tomatoes.append(tomato)
    return tomatoes

### Training data

#### Initialize the tomatoes for training data

In [4]:
tomatoes=['SingleImage','SingleImage1']
print(tomatoes)
X_train, y_train =  getTomatoes(tomatoes, 'Training', print_n=True, k_fold=False)
X_test, y_test = getTomatoes(tomatoes, 'Testing', print_n=True, k_fold=False)

['SingleImage', 'SingleImage1']
There are  2   Training images of  SingleImage
There are  2   Training images of  SingleImage1
There are  2   Testing images of  SingleImage
There are  2   Testing images of  SingleImage1


#### Use K-fold for training data

In [5]:
X,y = getTomatoes(tomatoes, '', print_n=True, k_fold=True)
print("X = ",X)
print("y = ",y)

X =  [[[[186 112  99]]]


 [[[171 143 129]]]


 [[[199  60  37]]]


 [[[150 112  91]]]


 [[[182  73  53]]]


 [[[121 113  66]]]


 [[[119  46  31]]]


 [[[187 180 154]]]]
y =  [0 0 1 1 0 0 1 1]


#### Before using standard scaler 

In [6]:
print(X_train[0].shape)

(1, 1, 3)


In [7]:
print(X_test[0].shape)

(1, 1, 3)


#### Scaling the images using Standard Scaler for training data

In [8]:
scaler = StandardScaler()
X_train1 = scaler.fit_transform([i.flatten() for i in X_train])
X_test = scaler.fit_transform([i.flatten() for i in X_test])
X = scaler.fit_transform([i.flatten() for i in X])
print("X = ",X)

X =  [[ 0.74837099  0.17179354  0.40190144]
 [ 0.22926972  0.91924615  1.13263134]
 [ 1.19825875 -1.08199793 -1.10827368]
 [-0.49747204  0.17179354  0.20704014]
 [ 0.60994398 -0.76855006 -0.71855107]
 [-1.50106782  0.19590492 -0.40190144]
 [-1.57028132 -1.41955717 -1.25441966]
 [ 0.78297774  1.81136701  1.74157292]]


In [9]:
print(X_train[0].shape)

(1, 1, 3)


In [10]:
print(X_test[0:1].shape)

(1, 3)


In [11]:
print(y_train.shape)

(4,)


In [12]:
print(y_test.shape)

(4,)


In [13]:
def plot_image_grid(images, nb_rows, nb_cols, figsize=(15,15)):
    assert len(images) == nb_rows*nb_cols, "Number of images should be the same as (nb_rows*nb_cols)"
    n = 0
    fig, axes = plt.subplots(nb_rows,nb_cols, figsize=figsize)
    for i in range(0, nb_rows):
        for j in range(0, nb_cols):
            axes[i, j].axis('off')
            axes[i, j].imshow(images[n])
            n += 1   

### 1) Trained Mobile Captured Single Image

In [14]:
print(tomatoes)
print(tomatoes[y_train[1]])
# plot_image_grid(X_train[0:1],1,1)

['SingleImage', 'SingleImage1']
SingleImage


### 1) Using Linear SVM for Classification on trained data

##### Define the model and evaluate all the metrics

In [15]:
tomatoes = ['SingleImage','SingleImage1']

#Get Images and Labels
X, y =  getTomatoes(tomatoes, 'Training')
X_test, y_test = getTomatoes(tomatoes, 'Testing')

#Scale Data Images
scaler = StandardScaler()
X_train = scaler.fit_transform([i.flatten() for i in X])
X_test = scaler.fit_transform([i.flatten() for i in X_test])

In [16]:
# Linear SVM
model = SVC(gamma='scale', kernel='linear',probability=True)
model.fit(X_train, y) 
y_predict = model.predict(X_test)
print("Y_Predict =",y_predict)

Y_Predict = [0 1 1 0]


### 2) Kernel RBF SVM for trained data

In [17]:
# Kernel RBF SVM
model = SVC(gamma='scale', kernel='rbf',probability=True)
model.fit(X_train, y) 
y_predict = model.predict(X_test)
print("Y_Predict =",y_predict)

Y_Predict = [0 1 1 0]


### 3) KNN for trained data

In [18]:
model = KNeighborsClassifier(n_neighbors=1,weights='uniform',metric='minkowski')
model.fit(X_train, y) 
y_predict = model.predict(X_test)
print("Y_Predict =",y_predict)

Y_Predict = [0 1 1 0]


## 4) Using Logistic Regression on the training data

In [19]:
from sklearn.linear_model import LogisticRegression

In [20]:
model = LogisticRegression(random_state=10,n_jobs=1,)
model.fit(X_train, y) 
y_predict = model.predict(X_test)
print("Y_Predict =",y_predict)

Y_Predict = [1 1 1 0]


## 5) Using ANN on the training data

In [21]:
from sklearn.neural_network import MLPClassifier

In [22]:
model = MLPClassifier(hidden_layer_sizes=(10,),activation='relu',solver='adam',alpha=0.0001,batch_size='auto',
                      learning_rate='constant',learning_rate_init=0.001,random_state=10)
model.fit(X_train, y) 
y_predict = model.predict(X_test)
print("Y_Predict =",y_predict)

Y_Predict = [1 1 1 0]


### Testing data

In [23]:
def getTomatoes(tomatoes, data_type, print_n=False, k_fold=False):
    images = []
    labels = []
    val = ['Testing']
    if not k_fold:
        path = "C://Users/Ishan.Borkar/Desktop/Tomato analysis/Kaggle/" + data_type + "/"
        for i,f in enumerate(tomatoes):
            p = path + f
            j=0
            for image_path in glob.glob(os.path.join(p, "*.jpg")):
                image = cv2.imread(image_path, cv2.IMREAD_COLOR)
                image = cv2.resize(image, (dim, dim))
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
                images.append(image)
                labels.append(i)
                j+=1
            if(print_n):
                print("There are" , j , "" , data_type, "images of" , tomatoes[i])
        images = np.array(images)
        labels = np.array(labels)
        return images, labels
    else:
        for v in val:
            path = "C://Users/Ishan.Borkar/Desktop/Tomato analysis/Kaggle/" + v + "/"
            for i,f in enumerate(tomatoes):
                p = path + f
                j=0
                for image_path in glob.glob(os.path.join(p, "*.jpg")):
                    image = cv2.imread(image_path, cv2.IMREAD_COLOR)
                    image = cv2.resize(image, (dim, dim))
                    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
                    images.append(image)
                    labels.append(i)
                    j+=1
        images = np.array(images)
        labels = np.array(labels)
        return images, labels
    
def getAllTomatoes():
    tomatoes = []
    for tomato_path in glob.glob("C://Users/Ishan.Borkar/Desktop/Tomato analysis/Kaggle/Testing/*"):
        tomato = tomato_path.split("/")[-1]
        tomatoes.append(tomato)
    return tomatoes

#### Initialize the tomatoes for testing data

In [24]:
tomatoes=['SingleImage','SingleImage1']
print(tomatoes)
X_train, y_train =  getTomatoes(tomatoes, 'Testing', print_n=True, k_fold=False)
X_test, y_test = getTomatoes(tomatoes, 'Training', print_n=True, k_fold=False)

['SingleImage', 'SingleImage1']
There are 2  Testing images of SingleImage
There are 2  Testing images of SingleImage1
There are 2  Training images of SingleImage
There are 2  Training images of SingleImage1


#### Use k-fold for testing data

In [25]:
X,y = getTomatoes(tomatoes, '', print_n=True, k_fold=True)
print("X = ",X)
print("y = ",y)

X =  [[[[182  73  53]]]


 [[[121 113  66]]]


 [[[119  46  31]]]


 [[[187 180 154]]]]
y =  [0 0 1 1]


#### Using standard scalar for testing data

In [26]:
scaler = StandardScaler()
X_train1 = scaler.fit_transform([i.flatten() for i in X_train])
X_test = scaler.fit_transform([i.flatten() for i in X_test])
X = scaler.fit_transform([i.flatten() for i in X])
print("X = ",X)

X =  [[ 0.92087722 -0.59473025 -0.49209824]
 [-0.967308    0.19824342 -0.21395576]
 [-1.02921571 -1.12998747 -0.9628009 ]
 [ 1.0756465   1.5264743   1.6688549 ]]


### 1) Tested Mobile Captured Single Image

In [27]:
print(tomatoes[y_train[0]])
# plot_image_grid(X_train[0:1], 1, 1)

SingleImage


### Multiclass classification for testing data

In [28]:
tomatoes = ['SingleImage','SingleImage1']

#Get Images and Labels
X, y =  getTomatoes(tomatoes, 'Testing')
X_test, y_test = getTomatoes(tomatoes, 'Training')

#Scale Data Images
scaler = StandardScaler()
X_train = scaler.fit_transform([i.flatten() for i in X])
X_test = scaler.fit_transform([i.flatten() for i in X_test])

### 1) Using Linear SVM for testing data

In [29]:
model = SVC(gamma='scale', kernel='linear',probability=True)
model.fit(X_train, y) 
y_predict = model.predict(X_test)
print("Y_Predict =",y_predict)

Y_Predict = [0 1 0 0]


### 2) Using Kernel RBF SVM for testing data

In [32]:
model = SVC(gamma='scale', kernel='rbf',probability=True)
model.fit(X_train, y) 
y_predict = model.predict(X_test)
print("Y_Predict =",y_predict)

Y_Predict = [0 1 0 0]


### 3) Using K-NN for testing data 

In [33]:
model = KNeighborsClassifier(n_neighbors=1,weights='uniform',metric='minkowski')
model.fit(X_train, y) 
y_predict = model.predict(X_test)
print("Y_Predict =",y_predict)

Y_Predict = [0 1 0 0]
